# Data Parse

In [2]:
import pandas as pd
import ast 

#### Columns as Lists with Dicts to extract data from

In [3]:
cols = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries',
        'spoken_languages', 'Keywords', 'cast', 'crew']

In [4]:
train_df = pd.read_csv('train.csv')
for col in cols:
    print(col)
    out_df = pd.DataFrame()  # empty dataframe to fill
    train_df[col] = train_df[col].fillna('[]')
    for i in range(len(train_df)):
        movie_id = int(train_df.loc[i, 'id'])  # keep track of movie's id
        data = ast.literal_eval(train_df.loc[i, col])
        if len(data) > 0:
            for j in range(len(data)):
                row_dict = {'movie_id': movie_id}
                row_dict.update(data[j])
                out_df = out_df.append(row_dict, ignore_index=True)
    out_df.to_csv('train_' + col + '.csv', index=False) 

belongs_to_collection


In [ ]:
test_df = pd.read_csv('test.csv')

for col in cols:
    print(col)
    out_df = pd.DataFrame()  # empty dataframe to fill
    test_df[col] = test_df[col].fillna('[]')
    for i in range(len(test_df)):
        movie_id = int(test_df.loc[i, 'id'])  # keep track of movie's id
        data = ast.literal_eval(test_df.loc[i, col])
        if len(data) > 0:
            for j in range(len(data)):
                row_dict = {'movie_id': movie_id}
                row_dict.update(data[j])
                out_df = out_df.append(row_dict, ignore_index=True)
    out_df.to_csv('test_' + col + '.csv', index=False)

# Data Cleanup

In [ ]:
# test to see if there are null values
train_df.isnull().values.any()

# select null values in dataframe
train_df[train_df.isnull().any(axis=1)]

# EDA

In [6]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.shape, test_df.shape

((3000, 23), (4398, 22))

In [43]:
train_genres_df = pd.read_csv('train_genres.csv')

print(train_genres_df.groupby(['name'])['id'].count().nlargest(10).reset_index(name='count'))
genre_freq_df = (train_genres_df
                     .groupby(['name'])['id']
                     .count()
                     .reset_index(name='count')
                     .sort_values(['count'], ascending=False)
                )

movie_genre_freq_df = train_genres_df.groupby(['movie_id'])['name'].count().reset_index(name='count')
movie_genre_freq_df.groupby('count').count().sort_values(['movie_id'], ascending=False)

              name  count
0            Drama   1531
1           Comedy   1028
2         Thriller    789
3           Action    741
4          Romance    571
5            Crime    469
6        Adventure    439
7           Horror    301
8  Science Fiction    290
9           Family    260


,movie_id
count,
2,972
3,900
1,593
4,393
5,111
6,21
7,3


In [15]:
train_key_df = pd.read_csv('train_Keywords.csv')

train_key_df.groupby(['name'])['id'].count().nlargest(10).reset_index(name='count')

,name,count
0,woman director,175
1,independent film,155
2,duringcreditsstinger,134
3,murder,123
4,based on novel,111
5,violence,87
6,sport,82
7,biography,77
8,aftercreditsstinger,75
9,dystopia,73


In [16]:
list_of_genres = list(train_df['genres'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
list_of_genres

TypeError: string indices must be integers

### Take a look at basic correlation from easily utilized Train data

In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

corr_df = train_df[train_df.budget > 0] #remove 0 values from budget
corr_df = corr_df[pd.notnull(corr_df['runtime'])] #remove null values from runtime

#### Correlation ####
# Correlation matrix with pandas
print(corr_df.corr())
# Individual correlation with numpy
print(np.corrcoef(corr_df.budget, corr_df.revenue)[0, 1]) # 0.7398
print(np.corrcoef(corr_df.popularity, corr_df.revenue)[0, 1]) # 0.4444
print(np.corrcoef(corr_df.runtime, corr_df.revenue)[0, 1]) # 0.2067

# Declare X and y
X = corr_df[['budget']] #, 'popularity', 'runtime']]
y = corr_df['revenue']

plt.scatter(X, y)
plt.title("Budget and Revenue")
plt.xlabel("Budget")
plt.ylabel("Revenue")
plt.show()

####################################################################################
# StatsModels
####################################################################################
# Add a constant
X = sm.add_constant(X)

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statisticsS
print(model.summary())

####################################################################################
# sklearn
####################################################################################
lm = skl.linear_model.LinearRegression()
model = lm.fit(X, y)
lm.score(X, y)
skl.feature_selection.f_regression(X, y)

### Take a look at the impact of the Cast

In [ ]:
train_cast_df = pd.read_csv('train_cast.csv')
train_cast_df = train_cast_df[['id', 'movie_id', 'order']]
train_cast_df.rename(columns={'id': 'person_id'}, inplace=True)

#### Merged Data Set ####
cast_df = pd.merge(train_df, train_cast_df, how='inner', on='movie_id')
# pivot is for an index without duplicates and pivot_table is the opposite
# print(cast_df.pivot(index='movie_id', columns='person_id', values='order'))
#print(cast_df.pivot_table(index='movie_id', columns='person_id', values='order', aggfunc='sum'))

test = cast_df.groupby(['movie_id', 'person_id'])['order'].sum().unstack()

test.dropna(axis=1)

test.to_csv('test.csv')